In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from matplotlib import rc
# import pandas as pd
# import seaborn as sns

%matplotlib inline

rc('animation', html='html5')
from ipywidgets import *
from IPython.display import display, clear_output
import trackpy as tp
import deeptrack as dt
from tensorflow import keras
from tensorflow.keras import backend as K
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

from sxmreader import SXMReader
# from analyzer import MotionAnalyzer
# from diffusion_plotter import DiffusionPlotter

2021-10-01 16:25:55.264777: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-01 16:25:55.264838: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
%%html
<style>
.box_bg {
    background-color: #003262;
}
</style>

In [4]:
class ImageSet:
    
    def __init__(self, folder, volt_temp, set_range, m_size, min_mass, sep, thresh, mthresh):
        self.folder = folder
        self.volt_temp = volt_temp
        self.set_range = set_range
        self.m_size = m_size
        self.min_mass = min_mass
        self.sep = sep
        self.thresh = thresh
        self.mthresh = mthresh
        self.SXM_PATH = [self.folder+"/Image_{0:03}.sxm".format(i) for i in self.set_range]
        self.frames = SXMReader(self.SXM_PATH)
        self.rawimgs = [self.frames.z_data[i] for i in range(len(self.frames))]
        self.masks = []
        dtdata = np.stack(self.rawimgs, axis=0)
        dtdata = np.expand_dims(dtdata[:], axis=-1)
        dtdata = dt.NormalizeMinMax(0, 1).resolve(list(dtdata))
        weighted_loss = dt.losses.flatten(dt.losses.weighted_crossentropy((10,1)))
        def iou_coef(y_true, y_pred, smooth=1):
          intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
          union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
          iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
          return iou
        model = keras.models.load_model(os.getcwd()+'/unet_model/iou-0.8610_N1_32-64-128-256_512_lossWL-BCE_bs32_nob60_lrsLIN_doREG.hdf5', custom_objects={'nd_unet_crossentropy': weighted_loss, 'iou_coef': iou_coef})
        predictions = model.predict(np.array(dtdata), batch_size=1)
        for i in range(len(predictions[:,:,:,0])):
            prediction = predictions[i,:,:,0] > self.mthresh
            self.masks.append(np.squeeze(prediction))
        self.batch = tp.batch(                        
            self.masks[:],
            self.m_size,
            minmass=self.min_mass,
            separation=self.sep,
            characterize=False,
            threshold=thresh,
            engine='python')
        
        
class PageSet():
    
    def __init__(self, cat, title, info, page):
        self.category = cat
        self.title_text = title
        self.info_text = info
        self.page = page
            

class AnalyzeUI(Box):
    
    def __init__(self):
        super().__init__()
        self.imset = []
        self.tempset = []
        self.voltstemps = []
        self.ranges = []
        self.pageindex = 0
        self.pages = []
        
        #Get a list of folders
        folder_list = [f.name for f in os.scandir() if f.is_dir() and f.name.startswith('electromigration')]
        
        output = Output(layout=Layout(
            width='420px',
            justify_content='center',
            align_items='center',
            padding='20px 10px 5px 5px'))
        
        with output:
            fig, ax = plt.subplots(figsize=(6.5,6.5))
            ln, = ax.plot([], [], lw=3)
            plt.axis('off')
            plt.show()
        
        def update(i):
            tp.annotate(
                self.tempset[0].batch[self.tempset[0].batch['frame']==i], 
                self.tempset[0].frames[i], 
                ax=ax)
            for line in ax.lines[:-1]:
                line.set_marker(None)
            return ln,
        
        def animate_plot(i):
            with output:
                output.clear_output(True)
                ani = FuncAnimation(
                    fig,
                    update,
                    frames=range(2), #int(np.ceil(len(self.tempset[0].frames)/2))
                    blit='True',
                    interval=300)
                display(HTML(ani.to_html5_video()))
        
        def f_style(text, size, color):
            return "<p style='font-size:{}'><font color='{}'>{}</p>".format(size, color, text)
        
        def bold(text):
            return '<b>{}</b>'.format(text)               
        
        def makedropdown(desc, opts, val, d=False, b_style='', tt=[],
            b='0px', h='auto', w='auto', jc='center', ai='center', ac='center'):
                return Dropdown(
                    description=desc,
                    options=opts,
                    value=val,
                    disabled=d,
                    button_style=b_style,
                    tooltips=tt,
                    style={'description_width': 'initial'},
                    layout=Layout(
                        height=h,
                        width=w,
                        justify_content=jc,
                        align_conten=ac,
                        align_items=ai,
                        border=b,
                        visibility='visible')
                    )
        
        def maketextbox(widget_type, val, desc=None,
            b='0px', h='auto', w='auto', p='auto', m='auto', jc='unset', ai='unset', ac='unset'):
                return widget_type(
                    description=desc,
                    value=val,
                    style={'description_width': 'initial'},
                    layout=Layout(
                        height=h,
                        width=w,
                        padding=p,
                        margin=m,
                        justify_content=jc,
                        align_conten=ac,
                        align_items=ai,
                        border=b,
                        visibility='visible'))
        
        def makebutton(desc, d=False, b_style='', b='0px', h='auto', w='auto'):
            return Button(
                description=desc,
                disabled=d,
                button_style=b_style,
                layout=Layout(border=b, height=h, width=w, visibility='visible'))
        
        def makeHTML(val, h='auto', w='auto', p='auto', m='auto', b='0px', jc='center', ai='center', ac='center'):
            return HTML(
                value=val,
                layout=Layout(
                    height=h,
                    width=w,
                    padding=p,
                    margin=m,
                    justify_content=jc,
                    align_conten=ac,
                    align_items=ai,
                    border=b,
                    visibility='visible'))
    
        def makebox(row_column, items,
            b='0px', h='auto', w='auto', p='auto', m='auto', jc='center', ai='center', ac='center'):
                return Box(
                    items,
                    layout=Layout(
                        display='flex',
                        flex_flow=row_column,
                        height=h,
                        width=w,
                        padding=p,
                        margin=m,
                        justify_content=jc,
                        align_conten=ac,
                        align_items=ai,
                        border=b,
                        visibility='visible'))
            
        def makepage(page_box):
            t_title.value = self.pages[self.pageindex].title_text
            t_info.value = self.pages[self.pageindex].info_text
            b_body = makebox('row', [page_box], h='425px', w='100%')
            return makebox('column', [b_header, b_body, b_footer], h='600px', w='100%', b='2px solid gray')
            
        def update_text(var, text, size, error='No'):
            if var == 'info' and error == 'No':
                t_info.value = f_style(text, size, 'white')
            elif var == 'info' and error =='Yes':
                t_info.value = f_style(text, size, 'red')
            else:
                t_title.value = f_style(bold(text), size, '#FDB515')
            
        def loadparams(dn):
            if dn == 'defaults':
                i_msize.value = str(param_defaults[0])
                i_minmass.value = str(param_defaults[1])
                i_sep.value = str(param_defaults[2])
                i_thresh.value = str(param_defaults[3])
                i_mthresh.value = str(param_defaults[4])
            else:
                i_msize.value = str(self.imset[self.pageindex-1].m_size)
                i_minmass.value = str(self.imset[self.pageindex-1].min_mass)
                i_sep.value = str(self.imset[self.pageindex-1].sep)
                i_thresh.value = str(self.imset[self.pageindex-1].thresh)
                i_mthresh.value = str(self.imset[self.pageindex-1].mthresh)

        def anchor_noanchor(choice):
            if choice == 'Yes':
                yes_text = 'This is the yes selection!'
                yesanchor = PageSet(
                    cat='yesanchor',
                    title=f_style(bold('Anchor Selection'), '32px', '#FDB515'),
                    info=f_style(yes_text, '18px', 'white'),
                    page=sb_anchor
                    )
                self.pages.append(yesanchor)
            else:
                no_text = 'NO ANCHORS HERE!'
                noanchor = PageSet(
                    cat='noanchor',
                    title=f_style(bold('No Anchor!'), '32px', '#FDB515'),
                    info=f_style(no_text, '18px', 'white'),
                    page=sb_anchor
                    )
                self.pages.append(noanchor)
                
                
        def current_page(p):
            print('page index: {}'.format(self.pageindex))
            if p == 'params':
                with output:
                    output.clear_output(True)
                    display(HTML('<img src="loading3.gif">'))
                self.tempset.clear()
                loadparams('defaults')
                if len(self.imset) >= self.pageindex:
                    with output:
                        output.clear_output(True)
                        display(HTML('<img src="loading3.gif">'))
                    self.tempset.append(self.imset[self.pageindex-1])
                    loadparams('saved')
                else:
                    with output:
                        output.clear_output(True)
                        display(HTML('<img src="loading3.gif">'))
                    tp.quiet()
                    self.tempset.append(ImageSet(
                        str(i_folders.value),
                        self.voltstemps[self.pageindex-1],
                        self.ranges[self.pageindex-1],
                        int(i_msize.value),
                        float(i_minmass.value),
                        float(i_sep.value),
                        float(i_thresh.value),
                        float(i_mthresh.value)))
                animate_plot(self.pageindex-1)
                param_title = '{} at {:.2f}'.format(i_folders.value, self.voltstemps[self.pageindex-1])
                self.pages[self.pageindex].title_text = f_style(bold(param_title), '32px', '#FDB515')
                self.children = [makepage(self.pages[self.pageindex].page)]
            elif p == 'anchor':
                print(len(self.pages))
                self.children = [makepage(self.pages[self.pageindex].page)]
            else:
                self.children = [makepage(self.pages[self.pageindex].page)]
            i_footer_btn1.disabled = False
            i_footer_btn2.disabled = False
        
        def change_param(change):
            with output:
                output.clear_output(True)
                display(HTML('<img src="loading3.gif">'))
            self.tempset.clear()
            tp.quiet()
            self.tempset.append(ImageSet(
                str(i_folders.value),
                self.voltstemps[self.pageindex-1],
                self.ranges[self.pageindex-1],
                int(change.new) if change['owner'].description == i_msize.description else int(i_msize.value),
                float(change.new) if change['owner'].description == i_minmass.description else float(i_minmass.value),
                float(change.new) if change['owner'].description == i_sep.description else float(i_sep.value),
                float(change.new) if change['owner'].description == i_thresh.description else float(i_thresh.value),
                float(change.new) if change['owner'].description == i_mthresh.description else float(i_mthresh.value)))
            animate_plot(self.pageindex-1)
        
        def click_back(b):
            print('pressed back')
            i_footer_btn1.disabled = True
            i_footer_btn2.disabled = True
            update_text('info', 'Loading, please wait...', '18px')
            if self.pageindex == 1:                
                self.pageindex += -1
                self.children = [makepage(self.pages[0].page)]
                i_footer_btn1.disabled = True
                i_footer_btn2.disabled = False
            else:
                if self.pageindex > len(self.voltstemps)+1:
                    self.pages.pop()
                self.pageindex += -1
                current_page(self.pages[self.pageindex].category)
        
        def click_next(b):
            print('pressed next')
            i_footer_btn1.disabled = True
            i_footer_btn2.disabled = True
            update_text('info', 'Loading, please wait...', '18px')
            if i_folders.value == None:
                    text = 'PLEASE SELECT A FOLDER AND TRY AGAIN'
                    update_text('info', text , '18px', error='Yes')
            elif self.pageindex == 0:
                self.voltstemps = np.linspace(
                    float(i_vt_initial.value),
                    float(i_vt_final.value),
                    int(i_vt_n.value),
                    dtype=np.float32)
                self.ranges = [range(
                    i_im_start.value+i_im_batch.value*i,
                    i_im_start.value+i_im_batch.value*(i+1)
                    ) for i in range(len(self.voltstemps))]
                if len(self.pages) <= 1:
                    p_title = '{} at {:.2f}'.format(i_folders.value, self.voltstemps[self.pageindex])
                    p_info1 = 'Confirm accuracy of tracking, adjust parameters if necessary,'
                    p_info2 = ' and click Next to continue.'
                    p_params = PageSet(
                        cat='params',
                        title=f_style(bold(p_title), '32px', '#FDB515'),
                        info=f_style(p_info1+p_info2, '18px', 'white'),
                        page=sb_params)
                    for i in range(len(self.voltstemps)):
                        self.pages.append(p_params)
                    self.pages.append(p_anchor)
                self.pageindex += 1
                current_page(self.pages[self.pageindex].category)
            else:
                if self.pageindex <= len(self.imset):
                    self.imset.append(self.tempset[0])
                elif self.pageindex > len(self.voltstemps):
                    anchor_noanchor(str(i_anchor.value))                    
                    print(str(i_anchor.value))
                self.pageindex += 1
                current_page(self.pages[self.pageindex].category)
            i_footer_btn1.disabled = False
            i_footer_btn2.disabled = False
                               
        # define input widgets
        msize_options = ['3', '5', '7', '9', '11']
        minmass_options = ['1.6', '1.8', '2.0', '2.2', '2.4']
        sep_options = ['1.0', '1.5', '2.0', '2.5', '3.0']
        thresh_options = ['0.1', '0.06', '0.05', '0.025', '0.003']
        mthresh_options = ['0.91', '0.93', '0.95', '0.97', '0.99'] 
        param_defaults = ['3', '1.7', '2.0', '0.06', '0.99']
        
        i_footer_btn1 = makebutton('Back', b_style='primary', h='50px', w='100px', d=True)
        i_footer_btn2 = makebutton('Next', b_style='danger', b='1px solid black', h='50px', w='100px')
        
        i_msize = makedropdown('Molecule Size: ', msize_options, '3', w='150px')
        i_minmass = makedropdown('Minimum Mass: ', minmass_options, '1.8', w='160px')
        i_sep = makedropdown('Separation: ', sep_options, '2.0', w='140px')
        i_thresh = makedropdown('TrackPy Threshold: ', thresh_options, '0.06', w='190px')
        i_mthresh = makedropdown('DeepTrack Threshold: ', mthresh_options, '0.99', w='200px')
        i_folders = Dropdown(options=folder_list, value=None, layout=Layout(width='180px'))
        i_anchor = Dropdown(options=['Yes','No'], value='No', layout=Layout(width='180px'))
        
        i_vt_initial = maketextbox(FloatText, 0.4, desc='Initial Value: ', w='140px')
        i_vt_final = maketextbox(FloatText, 0.54, desc='Final Value: ', w='140px')
        i_vt_n = maketextbox(FloatText, 8, desc='Number of Values: ', w='180px')
        i_im_start = maketextbox(IntText, 395, desc='Starting Image: ', w='170px')
        i_im_batch = maketextbox(IntText, 15, desc='Batch Size: ', w='140px')
        
        # define text widgets
        anchor_text = 'Does this set contain an anchor?'
        
        t_title = makeHTML('', jc='center')
        t_info = makeHTML('')        
        t_text = makeHTML(f_style(bold(anchor_text), '18px', 'black'), p='5px 5px 20px 5px')
        t_params = makeHTML(f_style(bold('Parameters'), '18px', 'black'), p='5px 5px 20px 5px')
        t_folders = makeHTML(bold('Folder Name: '))
        
        # define sub-boxes
        bgrey = '1px dashed grey'
        l_footer_btns = [i_footer_btn1, i_footer_btn2]
        l_folders = [t_folders, i_folders]
        l_voltstemps = [i_vt_initial, i_vt_final, i_vt_n]
        l_images = [i_im_start, i_im_batch]
        l_param = [i_msize, i_minmass, i_sep, i_thresh, i_mthresh]
        
        sb_info = makebox('row', [t_info], h='70px', w='725px')
        sb_footer_btns = makebox('row', l_footer_btns, h='70px', w='220px', ai='center', jc='space-around')            
        sb_folders = makebox('row', l_folders, w='295px', jc='space-around')
        sb_voltstemps = makebox('column', l_voltstemps, h='130px', w='210px', jc='space-around', b=bgrey)
        l_voltstemps_group = [makeHTML(bold('Votages/Temperatures: ')), sb_voltstemps]
        sb_voltstemps_group = makebox('row', l_voltstemps_group, w='415px', p='5px 25px 5px 5px')   
        sb_images = makebox('column', l_images, h='80px', w='190px', jc='space-around', b='1px dashed black')
        l_images_group = [makeHTML(bold('Image Frames: ')), sb_images]
        sb_images_group = makebox('row', l_images_group, w='320px', p='5px 0px 5px 5px')
        sb_param = makebox('column', l_param, h='250px', w='300px', jc='space-around', ai='center', b=bgrey)
        l_params_group = [t_params, sb_param]
        sb_params_group = makebox('column', l_params_group, m='5px 5px 25px 80px')
        sb_anchor = makebox('column', [i_anchor], h='70px', w='210px', jc='space-around', ai='center', b=bgrey)
        l_anchor_group = [t_text, sb_anchor]
        sb_anchor_group = makebox('column', l_anchor_group, m='5px 5px 25px 80px')
        sb_plot = makebox('column', [output], h='385px', w='425px')
        
        # define each page
        l_initial = [sb_folders, sb_voltstemps_group, sb_images_group]
        sb_initial = makebox('column', l_initial, h='325px', w='100%')
        l_params = [sb_params_group, sb_plot]
        sb_params = makebox('row', l_params, h='395px', w='100%', jc='center')
        sb_anchor = makebox('row', [sb_anchor_group], h='395px', w='100%', jc='center')
        
        # define main boxes for ui (these do not change)
        b_header = makebox('row', [t_title], h='100px', w='100%', b='1px solid gray', jc='center', ai='center')
        b_header.add_class('box_bg')
        b_footer = makebox('row', [sb_info, sb_footer_btns], h='75px', w='100%', b='1px solid gray')
        b_footer.add_class('box_bg')
        
        # observe widgets
        i_footer_btn1.on_click(click_back)
        i_footer_btn2.on_click(click_next)
        i_msize.observe(change_param, 'value')
        i_minmass.observe(change_param, 'value')
        i_sep.observe(change_param, 'value')
        i_thresh.observe(change_param, 'value')
        i_mthresh.observe(change_param, 'value')
        
        initial_text = 'Select a folder, provide the experiment information, and click next'
        self.pages.append(PageSet(
            cat='initial',
            title=f_style(bold('ANALYZE UI Mk.4'), '32px', '#FDB515'),
            info=f_style(initial_text, '18px', 'white'),
            page=sb_initial
            ))
        anchor_text = 'After making a selection, press next to continue.'
        p_anchor = PageSet(
            cat='anchor',
            title=f_style(bold('Anchor Selection'), '32px', '#FDB515'),
            info=f_style(anchor_text, '18px', 'white'),
            page=sb_anchor
            )
        
#         self.children = [makepage(sb_params)]
        self.children = [makepage(self.pages[0].page)]
              
AnalyzeUI()

AnalyzeUI(children=(Box(children=(Box(children=(HTML(value="<p style='font-size:32px'><font color='#FDB515'><b…

pressed next
page index: 1


TraitError: Invalid selection: value not found

In [12]:
np.linspace(0, 14, 3, dtype=int)

array([ 0,  7, 14])